## **Import libraries**

In [1]:
import pandas as pd
import re
import numpy as np

## **Load corrupted file**

In [2]:
file_path = "/content/Dataset_Malawi_National_Football_Team_Matches_corruptd.csv"
df_raw = pd.read_csv(file_path, header=None, encoding='ISO-8859-1', engine='python', on_bad_lines='skip')

In [3]:
df_raw

,0,1,2,3,4,5
0,Date,Opponent,TeLmÛS²Ñeî¥ppoQent SnoÈÞ,ReGult,Venue,[ompe'ion
1,1957,"ÚNrthernæRhoeO8""ãâ",5,LÞss,UnRu,None
2,"wnp""Friendl<`Firs~I¸ternalioal)""",None,None,None,None,None
3,"08/02W3010""A¬geria",0,Æ,¯oss,Away,AFCON 2010&GÂoup Stage
4,d11/0Ó/20º0AngSa,0m2,"Los ®AwayØ""AtCOð 2010 ro4pèStage""",None,None,None
5,12/¬1<ûR10,wapda,2¢1,WçÙ,Himeóþiendly,None
6,9/06/2012,eAger×aß0,ä,³oss,Hme,"Worl4 Cup ØoalfJ3 \r\n08/09/ú012,þVana,a,,e..."
7,"ÊCO Qualfier""",None,None,None,None,None
8,13/10/20p_,Gha¤a,0,1,Los²,None
9,Hom­,AFCOê l alifier,None,None,None,None


In [5]:

def clean_cell(cell):
    if isinstance(cell, str):
        return re.sub(r'[^\x20-\x7E]', '', cell).strip()
    return cell

df_cleaned = df_raw.applymap(clean_cell)

<ipython-input-5-b54c5703c34a>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df_raw.applymap(clean_cell)


## **Merge columns into rows for parsing**

In [6]:
row_strings = df_cleaned.astype(str).agg(' '.join, axis=1).tolist()

## **Parse rows into structured format using regex**

In [9]:
pattern = re.compile(
    r'(?P<date>\d{1,2}[/\-]\d{1,2}[/\-]?\d{2,4}|\d{4})?[\s:"]*'
    r'(?P<team>[A-Za-z\s\-\']{3,})?[\s:"#]*'
    r'(?P<score1>\d+)?[\s;]*'
    r'(?P<score2>\d+)?[\s;]*'
    r'(?P<result>Win|Loss|Draw)?[\s;:]*'
    r'(?P<location>Home|Away)?[\s;:"]*'
    r'(?P<extra>.*?)?$',
    re.IGNORECASE
)


In [10]:

parsed_rows = []

for line in row_strings:
    match = pattern.search(line)
    if match:
        parsed = match.groupdict()
        if any(parsed.values()):
            parsed_rows.append(parsed)

## **Convert to DataFrame**

In [11]:
df_final = pd.DataFrame(parsed_rows)

## **Normalize**

In [12]:
df_final = df_final.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_final.dropna(how='all', inplace=True)

<ipython-input-12-ce975c1f408e>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_final = df_final.applymap(lambda x: x.strip() if isinstance(x, str) else x)


## **Fix casing**

In [15]:
print(df_final.columns)


Index(['Date', 'Team', 'Score1', 'Score2', 'Result', 'Location', 'Extra'], dtype='object')


In [16]:
df_final['Result'] = df_final['Result'].str.capitalize()
df_final['Location'] = df_final['Location'].str.capitalize()


In [17]:
df_final

,Date,Team,Score1,Score2,Result,Location,Extra
0,None,Date Opponent TeLmSeppoQent Sno ReGult Venue,None,None,None,None,[ompe'ion
1,1957,NrthernRhoeO,8,None,None,None,5 Lss UnRu None
2,None,wnp,None,None,None,None,"Friendl<`Firs~Iternalioal)"" None None None Non..."
3,None,None,08,None,None,None,"/02W3010""Ageria 0 oss Away AFCON 2010&Goup Stage"
4,None,None,None,None,None,None,"d11/0/200AngSa 0m2 LosAway""AtCO 2010 ro4pStage..."
5,None,None,12,None,None,None,/1<R10 wapda 21 W Himeiendly None
6,9/06/2012,eAgera,0,None,None,None,"oss Hme Worl4 Cup oalfJ308/09/012,Vana,a,,e2ss..."
7,None,CO Qualfier,None,None,None,None,None None None None None
8,13/10/20,None,None,None,None,None,p_ Ghaa 0 1 Los None
9,None,Hom AFCO l alifier None None None None,None,None,None,None,
